# 앱이 깔려있지 않은 사용자가 미리보기 페이지에 머물러 갔을 때 얼마만큼 설치율이 높아지는/낮아지는가

가설: 미리보기 페이지에는 사진과 제품에 대한 상세 정보가 더 많아서, 클릭 누르는 사람들의 의도가 더 명확하고 그렇기에 앱 미설치자에 대한 앱 설치율이 미리보기 페이지를 거치지 않은 사용자(이미지와 간단한 텍스트만 본 사용자)들에 비해 더 높을 것이다. 

### 통제 변인
1. FB 광고, Redirect 백그라운드(미리보기 페이지)가 있/없을때 => + 2가지
2. 이미지(있/없)+텍스트 있을때, 미리보기 페이지 백그라운드 있/없을때 => +4가지

### 실험 결과

특정 앱 기설치율은 redirect로 광고를 태워보면 대략적으로 아래와 같음을 알 수 있습니다.

$$해당\ 광고를\ 누른\ 사람들의\ 앱\ 기설치율 = 1 - { 설치\ event\ 개수(104번) + 이탈\ 개수(202번-101번)  \over 전체\ redirect\ event\ 개수(202번)}$$

기설치율을 가지고, 미리보기 페이지를 거쳐갔을때 미설치자에 대한 설치율을 계산합니다.

$$미리보기\ 페이지\ 거쳐갔을때\ 설치율 = {(설치\ event\ 개수) \over (로드\ event\ 개수) - (도달\ event\ 개수) - (이탈\ 개수)*기설치율}$$

미리보기 페이지를 거치지 않았을때 미설치자에 대한 설치율을 계산합니다.

$$redirect로 바로 갔을때\ 설치율 = {(설치\ event\ 개수) \over (전체\ redirect\ event\ 개수) - (도달\ event\ 개수)}$$

### Reference
https://en.wikibooks.org/wiki/LaTeX/Mathematics

In [377]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [378]:
def sql_to_df(sql_query):
    df = pd.read_sql(sql_query, engine)
    return df

In [333]:
numbers = """
7913
8966
8478
8976
8477
9240
9329
7836
8977
9241
6523
6529
6531
6532
6534
6535
6537
6538
6540
6543
"""

In [389]:
number = numbers.split("\n")
test_links = []
for idx, i in enumerate(number):
    if idx==0 or idx==len(number)-1:
        continue
    test_links.append("item"+str(i))
    #print "https://search.airbridge.io/mapssi/item"+str(i)

In [363]:
opt = {
  'db_host': 'airbridge.cwgovxcx5cmk.ap-northeast-1.rds.amazonaws.com'
}
DB_HOST = 'mysql://udl:xpgpfks!@{0}/udl?charset=utf8mb4'.format(opt['db_host'])

In [373]:
engine = create_engine(DB_HOST)

In [391]:
web_stats_query = """
SELECT 
link_id,
event_category,
transaction_id,
origin_url,
client_id,
client_ip,
user_agent,
social_referer_type,
target,
raw_deeplink,
requested_at
FROM tbl_stats_web 
WHERE ({0}) AND app_id=35
"""

In [392]:
test_links_query = []
for idx, link in enumerate(test_links):
    if idx+1 == len(test_links):
        test_links_query.append("link_id='"+link+"'")
    else:
        test_links_query.append("link_id='"+link+"' OR")
        
web_stats_query = web_stats_query.format(" ".join(test_links_query))

In [393]:
mobile_stats_query = """
SELECT 
web.id as web_id,
web.requested_at as web_requested_at,
mobile.id as mobile_id,
mobile.requested_at as mobile_requested_at,
web.event_category as web_event,  
mobile.event_category as mobile_event,
mobile.transaction_id as transaction_id,
mobile.app_id,
device_type,
link_id,
web.client_ip as web_client_ip,
mobile.client_ip as mobile_client_ip
FROM tbl_stats_web as web, tbl_stats_mobile as mobile 
WHERE ({0}) and mobile.transaction_id = web.transaction_id;
""".format(" ".join(test_links_query))

In [397]:
# web을 통해 들어온 모든 통계
link_stats_web = sql_to_df(web_stats_query)
link_stats_web

,link_id,event_category,transaction_id,origin_url,client_id,client_ip,user_agent,social_referer_type,target,raw_deeplink,requested_at
0,item6523,202,0de57578-da27-49dc-90c3-c08e649518d9,http://search.airbridge.io/mapssi/item6523,dc2d0b39-3b55-4044-b42c-bb9d40a1b881,39.7.56.38,"{\n ""browser"": {\n ""major"": ""47"", \n ""n...",,None,None,2015-12-28 03:02:51
1,item6523,202,a421236c-a8c3-4e1a-aa46-4a7b5c359d06,http://search.airbridge.io/mapssi/item6523,a421236c-a8c3-4e1a-aa46-4a7b5c359d06,211.249.40.10,"{\n ""browser"": {\n ""major"": ""45"", \n ""n...",,None,None,2015-12-29 15:42:42
2,item6523,202,05ce4dd0-551d-455a-9902-20105b3952e3,http://search.airbridge.io/mapssi/item6523,86cbdbc2-3a2e-4244-88b0-57e1956a9dd5,121.134.227.161,"{\n ""browser"": {\n ""major"": ""46"", \n ""n...",,None,None,2015-12-30 18:11:29
3,item6523,203,05ce4dd0-551d-455a-9902-20105b3952e3,http://search.airbridge.io/mapssi/item6523,None,121.134.227.161,"{\n ""browser"": {\n ""major"": ""46"", \n ""n...",,Android_Intent,intent://mapssi?item=6523&transactionId=05ce4d...,2015-12-30 18:11:30
4,item6523,202,40b1a19d-da07-497e-987a-a03668cc08e6,http://search.airbridge.io/mapssi/item6523,86cbdbc2-3a2e-4244-88b0-57e1956a9dd5,121.134.227.161,"{\n ""browser"": {\n ""major"": ""46"", \n ""n...",,None,None,2015-12-30 18:11:45
5,item6523,203,40b1a19d-da07-497e-987a-a03668cc08e6,http://search.airbridge.io/mapssi/item6523,None,121.134.227.161,"{\n ""browser"": {\n ""major"": ""46"", \n ""n...",,Android_Intent,intent://mapssi?item=6523&transactionId=40b1a1...,2015-12-30 18:11:45
6,item6523,202,543291d7-35b3-449b-9ae7-7d872ecc697c,https://search.airbridge.io/mapssi/item6523,543291d7-35b3-449b-9ae7-7d872ecc697c,66.249.77.119,"{\n ""browser"": {\n ""name"": ""Safari""\n }, ...",,None,None,2016-01-07 02:14:20
7,item6523,202,6317c178-bcd0-4e26-a80d-9ab9ca16e918,http://mapssi.airbridge.io/item6523,8d784b68-2d23-406c-b728-6ecdca879ee8,121.134.227.161,"{\n ""browser"": {\n ""major"": ""47"", \n ""n...",,None,None,2016-01-12 17:52:06
8,item6523,202,dab7565a-ab49-470b-b7d1-583159e0b8aa,https://mapssi.airbridge.io/item6523,543291d7-35b3-449b-9ae7-7d872ecc697c,66.249.64.82,"{\n ""browser"": {\n ""name"": ""Safari""\n }, ...",,None,None,2016-01-13 14:12:28
9,item6529,202,543291d7-35b3-449b-9ae7-7d872ecc697c,https://search.airbridge.io/mapssi/item6529,543291d7-35b3-449b-9ae7-7d872ecc697c,66.249.71.95,"{\n ""browser"": {\n ""name"": ""Safari""\n }, ...",,None,None,2016-01-06 23:45:59


In [399]:
link_stats_mobile = sql_to_df(mobile_stats_query)
link_stats_mobile

,web_id,web_requested_at,mobile_id,mobile_requested_at,web_event,mobile_event,transaction_id,app_id,device_type,link_id,web_client_ip,mobile_client_ip
0,255117,2015-12-30 18:11:29,10142,2015-12-30 18:11:31,202,101,05ce4dd0-551d-455a-9902-20105b3952e3,35,SM-N910K,item6523,121.134.227.161,121.134.227.161
1,255117,2015-12-30 18:11:29,10143,2015-12-30 18:11:39,202,103,05ce4dd0-551d-455a-9902-20105b3952e3,35,None,item6523,121.134.227.161,121.134.227.161
2,255118,2015-12-30 18:11:30,10142,2015-12-30 18:11:31,203,101,05ce4dd0-551d-455a-9902-20105b3952e3,35,SM-N910K,item6523,121.134.227.161,121.134.227.161
3,255118,2015-12-30 18:11:30,10143,2015-12-30 18:11:39,203,103,05ce4dd0-551d-455a-9902-20105b3952e3,35,None,item6523,121.134.227.161,121.134.227.161
4,255119,2015-12-30 18:11:45,10144,2015-12-30 18:11:46,202,101,40b1a19d-da07-497e-987a-a03668cc08e6,35,SM-N910K,item6523,121.134.227.161,121.134.227.161
5,255120,2015-12-30 18:11:45,10144,2015-12-30 18:11:46,203,101,40b1a19d-da07-497e-987a-a03668cc08e6,35,SM-N910K,item6523,121.134.227.161,121.134.227.161
6,252022,2015-12-28 03:29:54,9650,2015-12-28 03:29:55,202,101,10ae95b2-3b09-4a52-8c6d-6c882ce8e232,35,SM-N910K,item6531,121.134.227.161,121.134.227.161
7,252022,2015-12-28 03:29:54,9651,2015-12-28 03:30:02,202,103,10ae95b2-3b09-4a52-8c6d-6c882ce8e232,35,None,item6531,121.134.227.161,121.134.227.161
8,252023,2015-12-28 03:29:55,9650,2015-12-28 03:29:55,203,101,10ae95b2-3b09-4a52-8c6d-6c882ce8e232,35,SM-N910K,item6531,121.134.227.161,121.134.227.161
9,252023,2015-12-28 03:29:55,9651,2015-12-28 03:30:02,203,103,10ae95b2-3b09-4a52-8c6d-6c882ce8e232,35,None,item6531,121.134.227.161,121.134.227.161


### 앱 기설치율 계산

In [410]:
install_ratio = 0
link_stats_web[link_stats_web['event_category']==202].groupby(['client_ip', 'user_agent']).last()

link_id  \
client_ip                      user_agent                                                     
121.134.227.161                {\n  "browser": {\n    "major": "33", \n    "na...  item7913   
                               {\n  "browser": {\n    "major": "4", \n    "nam...  item7913   
                               {\n  "browser": {\n    "major": "46", \n    "na...  item6531   
                               {\n  "browser": {\n    "major": "47", \n    "na...  item8966   
                               {\n  "browser": {\n    "major": "7", \n    "nam...  item7913   
121.134.227.161, 66.249.82.199 {\n  "browser": {\n    "major": "8", \n    "nam...  item6538   
121.134.227.161, 66.249.82.99  {\n  "browser": {\n    "major": "27", \n    "na...  item6538   
127.0.0.1                      {\n  "browser": {\n    "major": "4", \n    "nam...  item7913   
                               {\n  "browser": {\n    "major": "47", \n    "na...  item7913   
                               {\n  "browser": {\n    "major": "7", \n    "nam...  item7913   
183.102.1.204                  {\n  "browser": {\n    "major": "47", \n    "na...  item6538   
211.249.40.10                  {\n  "browser": {\n    "major": "45", \n    "na...  item6543   
211.249.40.21                  {\n  "browser": {\n    "major": "45", \n    "na...  item6531   
211.249.40.27                  {\n  "browser": {\n    "major": "45", \n    "na...  item6538   
218.146.74.166                 {\n  "browser": {\n    "major": "11", \n    "na...  item6538   
39.7.52.117                    {\n  "browser": {\n    "major": "47", \n    "na...  item6538   
39.7.54.183                    {\n  "browser": {\n    "major": "9", \n    "nam...  item7913   
39.7.56.38                     {\n  "browser": {\n    "major": "47", \n    "na...  item6523   
66.249.64.254                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6537   
66.249.64.82                   {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6523   
66.249.64.97                   {\n  "browser": {\n    "name": "Safari"\n  }, \...  item7836   
66.249.71.111                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item7836   
66.249.71.131                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item8966   
66.249.71.225                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6531   
66.249.71.234                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6535   
66.249.71.243                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6538   
66.249.71.252                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6537   
66.249.71.95                   {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6529   
66.249.74.103                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6532   
66.249.77.119                  {\n  "browser": {\n    "name": "Safari"\n  }, \...  item6534   
66.249.77.2                    {\n  "browser": {\n    "name": "Safari"\n  }, \...  item7913   

                                                                                   event_category  \
client_ip                      user_agent                                                           
121.134.227.161                {\n  "browser": {\n    "major": "33", \n    "na...             202   
                               {\n  "browser": {\n    "major": "4", \n    "nam...             202   
                               {\n  "browser": {\n    "major": "46", \n    "na...             202   
                               {\n  "browser": {\n    "major": "47", \n    "na...             202   
                               {\n  "browser": {\n    "major": "7", \n    "nam...             202   
121.134.227.161, 66.249.82.199 {\n  "browser": {\n    "major": "8", \n    "nam...             202   
121.134.227.161, 66.249.82.99  {\n  "browser": {\n    "major": "27", \n    "na...             202   
127.0.0.1                      {\n  "br

In [383]:
total_click = len(link_stats_web[link_stats_web['event_category']>=202])

In [384]:
link_stats_mobile = sql_to_df(mobile_stats_query)

In [385]:
total_reach = len(link_stats_mobile[link_stats_mobile['mobile_event']==101])

In [386]:
reach_after_install = len(link_stats_mobile[link_stats_mobile['mobile_event']==104])

In [387]:
unvalid_click = total_click - (total_reach - reach_after_install)

In [388]:
print unvalid_click

156


In [352]:
total_201_count = 0
total_203_count = 0
for row in link_stats_web.iterrows():
    platform = eval(row[1]['user_agent'])['platform']
    if platform != 'Desktop':
        if row[1]['event_category'] == 201:
            total_201_count = total_201_count + 1
        elif row[1]['event_category'] == 203:
            total_203_count = total_203_count + 1

    #if platform == 'iPhone':
    #    print platform, row[1]['link_id'], row[1]['event_category'], row[1]['transaction_id']
    #elif platform == 'Android':
    #    print platform, row[1]['link_id'], row[1]['event_category'], row[1]['transaction_id']

print "201 total count: " + str(total_201_count)
print "203 total count: " + str(total_203_count)

201 total count: 18
203 total count: 7


In [353]:
df_201 = link_stats_mobile[link_stats_mobile['mobile_event']==104][link_stats_mobile['web_event']==201]

In [354]:
df_203 = link_stats_mobile[link_stats_mobile['mobile_event']==104][link_stats_mobile['web_event']==203]

In [355]:
install_201_count = len(df_201)
install_203_count = len(df_203)

In [356]:
print "미리보기 페이지를 거쳐갔을때: " + str(float(install_201_count)/total_201_count*100) + "%"
print "바로 redirect 되었을때: " + str(float(install_203_count)/total_203_count*100) + "%"

미리보기 페이지를 거쳐갔을때: 22.2222222222%
바로 redirect 되었을때: 28.5714285714%
